# Overall Per-User Physiological Summary Across Stress, Aerobic, and Anaerobic Conditions

This section presents an overall per-user summary of wearable Electrodermal Activity (EDA) signals across stress, aerobic, and anaerobic sessions.
For each user, average EDA values are computed to compare physiological responses under different experimental conditions.


In [1]:
# basic libraries
import os
import pandas as pd
import numpy as np


In [3]:
# dataset main folder path
BASE_PATH = r"C:\Users\KARAN\Downloads\Internship Project\Wearable_Dataset\Wearable_Dataset"

# three condition folders
STRESS_PATH = os.path.join(BASE_PATH, "STRESS")
AEROBIC_PATH = os.path.join(BASE_PATH, "AEROBIC")
ANAEROBIC_PATH = os.path.join(BASE_PATH, "ANAEROBIC")

print("Paths loaded")
print("STRESS:", STRESS_PATH)
print("AEROBIC:", AEROBIC_PATH)
print("ANAEROBIC:", ANAEROBIC_PATH)


Paths loaded
STRESS: C:\Users\KARAN\Downloads\Internship Project\Wearable_Dataset\Wearable_Dataset\STRESS
AEROBIC: C:\Users\KARAN\Downloads\Internship Project\Wearable_Dataset\Wearable_Dataset\AEROBIC
ANAEROBIC: C:\Users\KARAN\Downloads\Internship Project\Wearable_Dataset\Wearable_Dataset\ANAEROBIC


In [4]:
# This function reads EDA.csv and returns overall mean and std
# It works even if start time is unix OR datetime string

def get_eda_summary(subject_folder_path):
    eda_path = os.path.join(subject_folder_path, "EDA.csv")
    
    # if EDA.csv not available, return None
    if not os.path.exists(eda_path):
        return None
    
    # read file
    eda_raw = pd.read_csv(eda_path, header=None)
    
    # EDA values start from row 3 onwards
    eda_values = eda_raw.iloc[2:, 0].astype(float).values
    
    # basic summary
    eda_mean = float(np.mean(eda_values))
    eda_std = float(np.std(eda_values))
    
    return eda_mean, eda_std


In [5]:
# This function loops over all subjects in one condition folder
# and collects overall EDA mean & std

def collect_condition_summary(condition_path, condition_name):
    rows = []
    
    # list all subject folders
    subjects = sorted([s for s in os.listdir(condition_path) if os.path.isdir(os.path.join(condition_path, s))])
    
    for subject in subjects:
        subj_path = os.path.join(condition_path, subject)
        
        result = get_eda_summary(subj_path)
        
        # if EDA missing, skip
        if result is None:
            continue
        
        eda_mean, eda_std = result
        
        rows.append([subject, condition_name, eda_mean, eda_std])
    
    return pd.DataFrame(rows, columns=["subject", "condition", "eda_mean", "eda_std"])


In [7]:
# collect summaries for each condition
stress_df = collect_condition_summary(STRESS_PATH, "stress")
aerobic_df = collect_condition_summary(AEROBIC_PATH, "aerobic")
anaerobic_df = collect_condition_summary(ANAEROBIC_PATH, "anaerobic")

# combine into one dataframe
overall_df = pd.concat([stress_df, aerobic_df, anaerobic_df], ignore_index=True)

print("Combined rows:", len(overall_df))
overall_df.head()


Combined rows: 100


,subject,condition,eda_mean,eda_std
0,S01,stress,0.643730,0.143060
1,S02,stress,1.149615,0.356382
2,S03,stress,2.725394,2.043679
3,S04,stress,1.097917,0.594482
4,S05,stress,14.095163,3.822214


In [8]:
# pivot table: one row per subject, columns are conditions
pivot_mean = overall_df.pivot_table(
    index="subject",
    columns="condition",
    values="eda_mean"
)

pivot_std = overall_df.pivot_table(
    index="subject",
    columns="condition",
    values="eda_std"
)

print("Overall per user (EDA mean):")
pivot_mean


Overall per user (EDA mean):


condition,aerobic,anaerobic,stress
subject,,,
S01,3.465261,6.071995,0.643730
S02,22.774827,8.261432,1.149615
S03,NaN,1.044729,2.725394
S03_,13.175080,NaN,NaN
S04,21.883998,3.343254,1.097917
S05,16.173826,41.501456,14.095163
S06,3.269664,NaN,3.708036
S06_,NaN,11.683999,NaN
S07,NaN,0.282745,0.257860


In [12]:
# CLEAN OVERALL SUMMARY TABLE
# One row per user, columns = stress/aerobic/anaerobic

pivot_mean = overall_df.pivot_table(
    index="subject",
    columns="condition",
    values="eda_mean"
)

# convert pivot to normal table
summary_small = pivot_mean.reset_index()

# round values to look clean
summary_small = summary_small.round(3)

# show only first 15 rows so it fits in one screenshot
summary_small.head(15)


condition,subject,aerobic,anaerobic,stress
0,S01,3.465,6.072,0.644
1,S02,22.775,8.261,1.150
2,S03,NaN,1.045,2.725
3,S03_,13.175,NaN,NaN
4,S04,21.884,3.343,1.098
5,S05,16.174,41.501,14.095
6,S06,3.270,NaN,3.708
7,S06_,NaN,11.684,NaN
8,S07,NaN,0.283,0.258
9,S07_,6.061,NaN,NaN


In [9]:
# save full long-format table
overall_df.to_csv("overall_per_user_summary.csv", index=False)
print("Saved: overall_per_user_summary.csv")

# save pivot mean table also (easier for report)
pivot_mean.to_csv("overall_per_user_EDA_mean.csv")
print("Saved: overall_per_user_EDA_mean.csv")


Saved: overall_per_user_summary.csv
Saved: overall_per_user_EDA_mean.csv


## Conclusion

In this project, we worked with wearable physiological data collected using the Empatica E4 device to analyze stress and exercise responses. We began by exploring the dataset to understand the signal structure and event timings. Using the provided event tags, we created stress and rest labels and then extracted simple statistical features from the Electrodermal Activity (EDA) signal using sliding windows.

We trained baseline machine learning models to distinguish between stress and rest states. Although the models were simple, the results showed that stress-related patterns can be identified from wearable EDA data. We also addressed class imbalance to ensure stable and fair evaluation of the models.

Finally, we performed an overall per-user analysis across stress, aerobic, and anaerobic sessions. This allowed us to compare physiological responses for each user under different conditions. The results showed clear differences between stress and exercise states, along with expected variability across individuals.

Overall, this project demonstrates how wearable sensor data can be processed and analyzed to study human stress and physical activity. 